## Thermal finite-element analysis for the BLAST CubeSat

This notebook shows how to use all modules in this repo for a reasonable preliminary on-orbit thermal analysis. 

We'll use a much more interesting (but still very simple) model of the sat. This one has six elements, one for each face of the sat. The sat's mass is evenly distributed among the elements, except the GGB tip mass is fully localized on the GGB face. The specific heat capacities still assume most of the mass is Al 6061 (which is pretty accurate, especially since most of our electronics are encased in that), except again for the tip mass, which is filled with lead shot. The long faces have solar panels on them, the GGB face exposed surfaces are entirely aluminum, and the antenna face only has the antenna's FR4 PCB exposed. Each face is connected to four adjacent faces. The conduction coefficient could definitely use some refining and I'm eyeballing the contact area, but I tried to be conservative in the hopes that internal radiation, which is not accounted for here, will act as additional effective thermal conduction in equalizing the different components of the sat.

All in all, this model should be quite accurate at reflecting the temperature range the sat's solar panels will experience. It is less accurate for internals, since the internal heat load of elecrtonic components will vary, so real components may experience temperature extremes outside of what this model shows.

In [1]:
from constants import *
from lumped_mass import *
from orbit import *
from rotation_sim import *
import numpy as np
import scipy

## Sat model

In [2]:
def init_blast_model(R0: np.ndarray|None = None, omega0: np.ndarray|None = None, omega_mag: float|None = np.radians(1)) -> tuple[list[ConnectedLumpedMass], Body, Orbit]:
    """Create lumped masses, a Body, and an Orbit to represent the BLAST CubeSat
    
    Uses six ConnectedLumpedMasses, one for each face.

    R0 can be optionally used to specify the starting body orientation in the inertial (stationary) frame.
        Randomized if not specified.
    omega0 can be optionally used to specify the starting body angular velocity in the body (rotating) frame.
        Randomized if not specified (with magnitude equal to omega_mag, default 1 deg/s).

    Creates an SGP4Orbit with ISS TLEs.
    """
    long_side_elt_dict = {
        'm': (CUBESAT_MASS-CUBESAT_TIP_MASS)/5,
        'A_rad': 10*u.cm*20*u.cm,
        'a': SOLAR_PANEL_ABSORPTANCE,
        'e': SOLAR_PANEL_EMISSIVITY,
        'q_int': CUBESAT_POWER_CONSUMPTION/5
    }

    elts = [ # long faces
        ConnectedLumpedMass(
            body_normal = np.array(normal),
            **long_side_elt_dict,
        )
    for normal in [[1,0,0], [0,1,0], [-1,0,0], [0,-1,0]]]

    elts.append(ConnectedLumpedMass( # GGB square face
        body_normal = np.array([0,0,-1]),
        m= (CUBESAT_MASS-CUBESAT_TIP_MASS)/10 + CUBESAT_TIP_MASS,
        c= PB_SPEC_HEAT_CAP,
        A_rad= 10*u.cm*10*u.cm + CUBESAT_TIP_MASS_EXT_AREA,
        a= AL_ABSORPTANCE,
        e= AL_EMISSIVITY,
        q_int= CUBESAT_POWER_CONSUMPTION/10
    ))
    elts.append(ConnectedLumpedMass( # antenna square face
        body_normal = np.array([0,0,1]),
        m= (CUBESAT_MASS-CUBESAT_TIP_MASS)/10,
        A_rad= 10*u.cm*10*u.cm,
        a= FR4_ABSORPTANCE,
        e= FR4_EMISSIVITY,
        q_int= CUBESAT_POWER_CONSUMPTION/10
    ))

    for i,long_face in enumerate(elts[:4]):
        # Each long face connects to both of the square faces
        long_face.connect(elts[4], K = AL_HEAT_CONDUCTIVITY * 2*u.cm * 10*u.cm)
        long_face.connect(elts[5], K = AL_HEAT_CONDUCTIVITY * 2*u.cm * 10*u.cm)
        # Each long face connects to two adjacent long faces,
        # but since connect is bidirectional, only call it on the next face
        long_face.connect(elts[(i+1)%4], K = AL_HEAT_CONDUCTIVITY * 2*u.cm * 20*u.cm)

    if omega0 is None:
        random_omega = np.random.normal(size=3)
        omega0 = random_omega * omega_mag/np.linalg.norm(random_omega) 
    if R0 is None:
        random_bodyR = scipy.stats.special_ortho_group.rvs(3)
        R0 = random_bodyR
    body = Body(
        np.diag([CUBESAT_ROTI_XY.to(u.kg*u.m**2).magnitude, CUBESAT_ROTI_XY.to(u.kg*u.m**2).magnitude, CUBESAT_ROTI_Z.to(u.kg*u.m**2).magnitude]),
        R0 = R0,
        omega0=omega0
    )

    orbit = SGP4Orbit(
        *ISS_TLE, 
        'excerpt_de440.bsp',
        t_tol=10
    )

    return elts, body, orbit

## Propagator

In [3]:
def eval_funct(t: float, state_vec: np.ndarray, t0: pd.Timestamp, body: Body, orbit: Orbit) -> np.ndarray:
    """Derivative function
    
    State vector convention: shape (9 + 3 + len(Body.elts)) in the order:
        R, omega, T

    Since solve_ivp doesn't suport
    Calculates Tdot for Body.elts using temperatures from y and orbital state from orbit,
    and calculates rotation matrix and angular velocity vector derivatives for Body.
    """ 
    body_R = state_vec[:9].reshape(3,3)
    body_omega = state_vec[9:12]
    T_vec = state_vec[12:]

    orbit.set_input(t)
    q_solar_normal, q_earth_normal = orbit.get_sun_earth_normal_power()
    r_earth, r_sun = orbit.get_earth_sun_vecs()

    body.set_input(body_R, body_omega)
    Rdot, omegadot = body.get_derivs()

    for T, elt in zip(T_vec, Body.elts):
        elt.set_input(t, T, r_earth, r_sun, body_R, q_solar_normal=q_solar_normal, q_earth_normal=q_earth_normal)
    Tdot_vec = np.array([elt.find_Tdot() for elt in Body.elts])

    return np.concat([Rdot, omegadot, Tdot_vec])

def propagate_thermal_fem(
        body: Body, orbit: Orbit, 
        t0: pd.Timestamp, t_end: pd.Timestamp, t_step: pd.Timedelta, 
        **kwargs
):
    """Propagate FEM with attitude, orbital position, and temperature.
    
    The numerical itegrator will be interrupted every t_step
    to reorthonormalize the body rotation martix
    and print a rudimentary progress log (since solve_ivp doesn't offer callbacks).
    """
    ts = []; ys = []; total_nfev = 0
    t= 0 # relative to start_t in sec
    y = np.concatenate((body.R.flatten(), body.omega, np.array([elt.T for elt in body.elts])))

    while t < (t_end-t0).total_seconds():
        res = scipy.integrate.solve_ivp(
            eval_funct, 
            (t, t+t_step.total_seconds()), 
            y, 
            args=(t0, body, orbit),
            **kwargs
        )
        assert res.status == 0
        ts.extend(res.t); ys.extend([res.y[:,i] for i in range(res.y.shape[1])]); total_nfev += res.nfev
        
        # Reorthonormalize rotation matrix
        body_R = res.y[:9, -1].reshape(3,3)
        body_R_corrected = reorthonormalize(body_R)

        t= res.t[-1]
        y = np.concatenate((body_R_corrected.flatten(), res.y[9:, -1]))

        if np.isclose(t % 100, 0): 
            print(f"{t} / {(t_end-t0).total_seconds()}, nfev: {total_nfev}")
            total_nfev = 0

    return ts, ys

In [5]:
elts, body, orbit = init_blast_model(R0=np.eye(3), omega0=np.array([0,0,np.radians(5)]))

In [14]:
def rotation_deriv(t, state_vec, body: Body):
    """Derivative function for Body rotation only.
    state_vec: shape (12,) = [R (9), omega (3)]
    Returns concatenated [Rdot.flatten(), omegadot]
    """
    R = state_vec[:9].reshape(3,3)
    omega = state_vec[9:12]
    body.set_input(R, omega)
    Rdot, omegadot = body.get_derivs()
    return np.concatenate([Rdot.flatten(), omegadot])

def propagate_rotation(body: Body, t_span, t_eval=None, **kwargs):
    """Propagate the rotation of a Body using solve_ivp.
    t_span: (t0, tf)
    t_eval: array of times to evaluate solution at (optional)
    Returns the result from solve_ivp.
    """
    y0 = np.concatenate([body.R.flatten(), body.omega])
    sol = scipy.integrate.solve_ivp(
        lambda t, y: rotation_deriv(t, y, body),
        t_span,
        y0,
        t_eval=t_eval,
        **kwargs
    )
    return sol